In [2]:
# import necessary modules 
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import netcdf
import math as math
import pandas as pd
from matplotlib import gridspec
import scipy
from scipy import stats, special


In [3]:
# Set Directories 
directory = '/glade/work/cmcclus/CAM_misc/cam_output/ForJayMace/'
filename = directory+'cam6_2032_sd_soar_base_MG3_01252023.cam.hs.2017_2018.merged.nc'
CAM = netcdf.netcdf_file(filename, 'r')

FileNotFoundError: [Errno 2] No such file or directory: '/glade/work/cmcclus/CAM_misc/cam_output/ForJayMace/cam6_2032_sd_soar_base_MG3_01252023.cam.hs.2017_2018.merged.nc'

In [ ]:
# EXTRACT VARIABLES FROM CAM NETCDF FILE
instr_num = CAM.variables['instr_num'][:] # orbit number (flight number)
orbit_num = CAM.variables['orbit_num'][instr_num==32] # orbit number (flight number)

lat = CAM.variables['lat'][instr_num==32] # latitude of CAM gridbox 
lon = CAM.variables['lon'][instr_num==32] # longitude of CAM gridbox
cam_time = CAM.variables['julian'][instr_num==32]-6575 # CAM timestamp; days since 2018-01-01 00:00:00

# correct time to hours (account for DOY)
CAM_time_corr_factor =0. #np.floor(np.min(cam_time))
time_adj = (cam_time+CAM_time_corr_factor) # convert timestamp to hours since 2015-07-17 00:00:00 to match observations. 

# hybrid grid-related variables 
P0 = CAM.variables['P0'][...] # reference pressure
PS = CAM.variables['PS'][instr_num==32] # surface pressure [=] Pa
hyam = CAM.variables['hyam'][...] # hybrid A coefficient at layer midpoints
hybm = CAM.variables['hybm'][...] # hybrid B coefficient at layer midpoints
P_dummy = P0*hyam
midP = (P_dummy+(hybm*PS[:,None]))# [=] Pa 
midP_hPa = midP*0.01 # [=] hPa
Z = CAM.variables['Z3'][instr_num==32] # geopotential Height  [=] m

# Met variables: 
RH = CAM.variables['RELHUM'][instr_num==32] # relative hum. [=] % 
Q = CAM.variables['Q'][instr_num==32] # specific humidity [=] kg/kg
T = CAM.variables['T'][instr_num==32] # K

# calc mass density 
Rdry = 287.
massdens = midP / T / Rdry # [=] kg/m3

# Cloud variables
CLOUD = CAM.variables['CLOUD'][instr_num==32] # cloud fraction [=] fraction
CLOUD[CLOUD<=1.e-4]=1.e-4

CLDLIQ = CAM.variables['CLDLIQ'][instr_num==32] # [=] kg/kg
CLDICE = CAM.variables['CLDICE'][instr_num==32] # [=] kg/kg
QRAIN = CAM.variables['QRAIN'][instr_num==32] # [=] kg/kg
QSNOW = CAM.variables['QSNOW'][instr_num==32] # [=] kg/kg

NUMLIQ = CAM.variables['NUMLIQ'][instr_num==32]*massdens# [=] #/kg conv. to #/m3
NUMICE = CAM.variables['NUMICE'][instr_num==32]*massdens# [=] #/kg conv. to #/m3
NRAIN = CAM.variables['NRAIN'][instr_num==32]# [=] #/m3
NSNOW = CAM.variables['NSNOW'][instr_num==32]# [=] #/m3

# converting grid-box average values to in-cloud values
icnc =  NUMLIQ/CLOUD# in cloud number concentration of liquid[=] #/m3
icqc = CLDLIQ/CLOUD # in cloud mixing ratio of liquid [=] kg/kg
 
icni = NUMICE/CLOUD # in cloud number concentration of ice [=] #/m3
icqi = CLDICE/CLOUD # in cloud mixing ratio of ice [=] kg/kg

icns = NSNOW/CLOUD # in cloud number concentration of snow [=] #/m3
icqs = QSNOW/CLOUD # in cloud mixing ratio of snow [=] kg/kg

icnr = NRAIN/CLOUD # in cloud number concentration of rain [=] #/m3
icqr = QRAIN/CLOUD # in cloud mixing ratio of rain [=] kg/kg

icqt = icqc+icqi+icqs+icqr


# if the CAM simulations report IWC or LWC lower than 4.68×10-5 g m-3, we set these variables equal to zero
minval = 4.68e-5*0.001 # minimum mass of ice/lquid [=] kg/m3

LWC = CAM.variables['LWC'][instr_num==32] # liquid water content [=] kg/m3
IWC = CAM.variables['IWC'][instr_num==32] # ice water content [=] kg/m3
# LWC[LWC<minval]=0.
# IWC[IWC<minval]=0.
TWC = LWC + IWC # total water content [=] kg/m3
Frac = LWC/TWC